In [1]:
%run utils.py
%matplotlib inline
import os.path as osp
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from unidecode import unidecode
pd.set_option('display.max_rows', 1000)

In [2]:
df_ct = pd.read_csv(osp.join(DATA_DIR, 'pubmed_abstract_tcell_types.csv'))
df_ct = df_ct[df_ct['type_key'].notnull()]
df_ct.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7045 entries, 0 to 11509
Data columns (total 6 columns):
type        7045 non-null object
count       7045 non-null int64
id          7045 non-null int64
type_key    7045 non-null object
type_lbl    7045 non-null object
type_lvl    7045 non-null float64
dtypes: float64(1), int64(2), object(3)
memory usage: 385.3+ KB


In [3]:
# Pull list of pubmed ids with a known cell type 
valid_ids = df_ct['id'].unique()
len(valid_ids)

6171

In [4]:
# Read and restrict protein mappings to only those for papers with associated
# cell type data (to limit number of proteins to resolve)
df = pd.read_csv(osp.join(DATA_DIR, 'pubmed_abstract_proteins.csv'))
df = df[df['id'].isin(valid_ids)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65161 entries, 1 to 453611
Data columns (total 5 columns):
id       65161 non-null int64
start    65161 non-null int64
end      65161 non-null int64
value    65161 non-null object
class    65161 non-null object
dtypes: int64(3), object(2)
memory usage: 3.0+ MB


### Split Protein Symbols

In [5]:
def is_sequence(text):
    # Examples:
    # CD127(low+/-) CD25(+) forkhead box protein 3
    # CD25(+)FoxP3(+)CD127(low)CD4(+)
    # CD8(+)CD28(-)T  
    # Foxp3(+)CD25(hi)CD127(lo)CD39(hi)
    # CD4⁺CD25⁺Fxop3⁺
    # FoxP3(+)RoRγt(+)IL17(+)
    
    # Counts of substrings that are possible in real words but upper case when used as protein name
    m_ct1 = sum([text.count(x) for x in ['CD', 'CCR', 'CCL', 'CXCR', 'ICOS', 'IL', 'TCR']])
    # Counts of very unique substrings in any case
    m_ct2 = sum(text.upper().count(x) for x in ['FOXP', 'STAT3'])
    # Counts of common separating characters 
    m_ct3 = sum([text.count(x) for x in ['lo', 'hi', 'Lo', 'Hi', '+', '(-)', '⁺', '⁻', '×']])
    return m_ct1 + m_ct2 >= 2 or m_ct3 >= 2

In [6]:
# Idenfity sequences of identifying proteins
df[df['value'].apply(is_sequence)]['value'].value_counts().head(150)

CD4(+)CD25(+)                                       356
CD4+CD25                                            343
CD4(+)CD25(-)                                       100
CD4(+)CD25(high)                                     82
CD4(+)CD25(+)Foxp3(+)                                80
CD4+CD25-                                            78
CD4(+) CD25(+)                                       73
CD4+CD25+Foxp3                                       48
CD4(+)CD25(+)FoxP3(+)                                47
CD4(+)Foxp3(+)                                       46
CD4+CD25high                                         35
CD4+CD25(high)                                       30
CD4+CD25+FoxP3                                       29
CD4+CD25+FOXP3                                       27
CD4(+)CD25(+)FOXP3(+                                 27
CD8+CD28-                                            25
CD4(+)CD25(hi)                                       25
CD4(+)FOXP3(+                                   

In [7]:
def split_compound(text):
    # Examples:
    # mmmCD4CXCR5CCR4CD62L -> ['CD4', 'CXCR5', 'CCR4', 'CD62L']
    parts = re.findall('(?:CD|CCR|CCL|CXCR|ICOS|FOXP|IL|TCR)\w+?(?=CD|CCR|CCL|CXCR|ICOS|FOXP|IL|TCR|$)', text.upper())
    parts = [p.strip() for p in parts]
    return [p for p in parts if len(p) > 0]

def split_sequence(text):
    # Note that the precedence here matters and that hyphen splits are left until the end since
    # they are very ambiguous and should come w/ relevant lookaheads
    parts = re.split('\\(.*?\\)|\\+|\\(|\\)|:|,|/|×|⁺|⁻|High|high|Low|low|Hi|hi|Lo|lo|-(?=CD)|-(?=CCR)|-(?=CXCR)|-(?=CCL)|-(?=IL)|-(?=TCR)|-$', text)
    parts = [p.strip() for p in parts]
    # Remove any single char elements or those with spaces (which are typically phrases)
    parts = [p for p in parts if len(p) > 1 and p.count(' ') == 0]
    
    # Lastly, now that all separating elements are gone, check each element to see if 
    # it is still a likely sequence and if so, apply splits based on naming conventions
    # (i.e. for cases like "CD4CXCR3CCR6" where there are no separating elements)
    res = []
    for p in parts:
        if not is_sequence(p):
            res.append(p)
            continue
        res.extend(split_compound(p))
        
    return res

In [8]:
# split_sequence('CD1highCD2 + CD1lowCD99(-)CD19')
# split_sequence('CD4(+)CD25(+)CCR4(+)')
# split_sequence('CD62LhighCD25')
# split_sequence('CD4+CD25+highFoxp3+CD62L+')
# split_sequence('IFN-γ(+) IL-4(-)')
# split_sequence('CD19+CD24+CD38+TGF-β1')
# split_sequence('Tissue inhibitor of metalloproteinase 1')
# split_sequence('CD45RA(-)CD45RO(+)CD95(hi)CD62L(lo) ')
# split_sequence('FALC Lin(-)c-Kit(+)Sca-1(+)')
# split_sequence('CD45RO+CD62Ll(ow)CCR7(low)CD40L(high)ICOS(high)')
split_sequence('CD45RO+CD62L-CD4-CXCR5')

['CD45RO', 'CD62L', 'CD4', 'CXCR5']

In [9]:
def melt_sequences(df):
    res = []
    for i, r in df.iterrows():
        is_seq = is_sequence(r['value'])
        if not is_seq:
            res.append(r)
            continue
        # Split sequence which may produce empty values
        values = split_sequence(r['value'])
        for v in values:
            # Write new value with all other fields as is
            rs = r.copy()
            rs['value'] = v
            res.append(rs)
    return pd.DataFrame(res)
df_mlt = melt_sequences(df)
df_mlt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69951 entries, 1 to 453611
Data columns (total 5 columns):
id       69951 non-null int64
start    69951 non-null int64
end      69951 non-null int64
value    69951 non-null object
class    69951 non-null object
dtypes: int64(3), object(2)
memory usage: 3.2+ MB


In [10]:
# Ideally, there should be no "sequences" left but the is_sequence method 
# intentionally aims for recall rather than precision while the split_sequences
# method does the opposite so there may be some remainder here and it should be
# validate manually as unimportant (and ideally small or empty)
df_mlt['value'][df_mlt['value'].apply(is_sequence)]

Series([], Name: value, dtype: object)

In [11]:
#df_mlt['value'].value_counts().head(250)

### Normalize Protein Symbols

In [12]:
# odds = 'αβγδκζÏ⁺⁻'
# unidecode(odds), unidecode(odds.upper())

In [26]:
import re
pr_m_sub = [    
    ('INTERFERON', 'IFN'),
    ('INTERLEUKIN', 'IL'),
    ('TRANSFORMINGGROWTHFACTOR', 'TGF'),
    ('TUMORNECROSISFACTOR', 'TNF'),
    ('TUMOURNECROSISFACTOR', 'TNF'),
    ('TCELLRECEPTOR', 'TCR'),
    ('FORKHEADBOXPROTEIN', 'FOXP'),
    ('FORKHEADBOXP', 'FOXP'),
    ('TOLLLIKERECEPTOR', 'TLR'),
    ('EOMESODERMIN', 'EOMES'),
    ('NLRP3CASPASE1', 'NLRP3'),
    ('MACROPHAGECOLONYSTIMULATINGFACTOR', 'MCSF'),
    ('TCELLREPLACINGFACTOR', 'IL5'),
    ('HUMANLEUKOCYTEANTIGEN', 'HLA'),
    ('RETINOICACIDRECEPTORRELATEDORPHANNUCLEARRECEPTOR', 'ROR'),
    ('RETINOICACIDRELATEDORPHANNUCLEAR', 'ROR'),
    ('RETINOICACIDRELATEDORPHAN', 'ROR'),
    ('ORPHANRETINOICACIDNUCLEAR', 'ROR'),
    ('RETINOIDACIDRELATEDORPHAN', 'ROR'),
    ('RORRELATEDORPHANNUCLEAR', 'ROR'),
    ('RETINOIDORPHANNUCLEAR', 'ROR'),
    ('RETINOIDRELATEDORPHAN', 'ROR'),
    ('RETINOICACIDORPHAN', 'ROR'),
    ('RETINOIDORPHAN', 'ROR'),
    ('RETINOICORPHAN', 'ROR'),
    ('RELATEDORPHAN', 'ROR'),
    ('RETINOICACID', 'ROR'),
    ('RETINOIDACID', 'ROR'),
    ('MAJORHISTOCOMPATIBILITYCOMPLEX', 'MHC'),
    ('SIGNALTRANSDUCERANDACTIVATOROFTRANSCRIPTION', 'STAT'),
    ('MONOCYTECHEMOTACTICPROTEIN1', 'MCP1'),
    ('INDOLEAMINE23DIOXYGENASE', 'IDO'),
    ('MHCCLASSII', 'MHCII'),
    ('HLACLASSII', 'MHCII'),
    ('CLASSIIMHC', 'MHCII'),
    ('MHCCLASSI', 'MHCI'),
    ('HLACLASSI', 'MHCI'),
    ('CLASSIMHC', 'MHCI'),
    ('MHCIRELATEDPROTEIN1', 'MHCI'),
    ('MHCRELATEDPROTEIN1', 'MHCI'),
    ('GRANZYME', 'GZM'),
    ('GRANZ', 'GZM'),
    ('ALPHA', 'A'),
    ('BETA', 'B'), 
    ('GAMMA', 'G'),
    ('DELTA', 'D'),
    ('KAPPA', 'K'),
    ('ZETA', 'Z'), 
    
    # Reordered-synonyms
    ('GIFN', 'IFNG'),
    ('GDTCR', 'TCRGD'), 
    
    # Final eliminations
    ('RECEPTORS', ''), ('RECEPTOR', ''), 
    ('LIGANDS', ''), ('LIGAND', ''),
    ('ANTIGENS', ''), ('ANTIGEN', ''),
    ('BRIGHT', ''), ('FAMILY', ''),
    ('CYTOKINES', ''), ('CYTOKINE', ''), ('ALARMINS', ''),
    ('CHEMOKINES', ''), ('CHEMOKINE', ''),
    ('CYTOKINES', ''), ('CYTOKINE', ''),
    ('LYMPHOKINES', ''), ('LYMPHOKINE', ''),
    ('RELATEDPROTEIN', '')
]
pr_m_lkp = dict([
    ('IGG', 'IgG'), ('IGA', 'IgA'), ('IGE', 'IgE'), ('IGD', 'IgD'), ('IGM', 'IgM'),
    ('PDL1', 'PD1'), ('SCD25', 'CD25'), ('CD8A', 'CD8'), ('CD8AA', 'CD8'),
    ('ILT2', 'CD85J'),
    ('LSELECTIN', 'CD62L'), 
    ('CD8SIRPA', 'SIRPA'),
    
    ('INTEGRINA4B7', 'INTEGRINB7'), 
    ('INTEGRINAEB7', 'INTEGRINB7'), 
    ('B2ORB7INTEGRIN', 'INTEGRINB7'),
    
    ('CCL4', 'MIP1B'), 
    
    ('GIFN', 'IFNG'),
    
    ('GDTCR', 'TCRGD'),
    ('TCRD', 'TCRGD'),
    ('DTCR', 'TCRGD'),
    ('TCRG', 'TCRGD'),
    ('GTCR', 'TCRGD'),
    
    ('TFGB', 'TGFB'),
    ('TGFB1', 'TGFB'),
    
    ('HVA72', 'VA72'),
    ('IVA72', 'VA72'), 
    ('TCRVA72', 'VA72'), 
    ('VA72JA33', 'VA72'),
    
    ('GALCER', 'AGALCER'),
    
    ('LAG3TREG', 'CD223'),
    ('PGD2SYNTHASE', 'PGD2'),
    ('LIPOCALINTYPEPGDSYNTHASE', 'PGD2'),
    ('NKG2ACD56', 'NKG2'), ('NKG2AC', 'NKG2'), ('NKG2D2B4', 'NKG2'),
    ('RIL9', 'IL9')
])
pr_m_bl = [
    'TRANSCRIPTIONFACTOR', 'TRANSCRIPTIONFACTORS', 'SUBOPTIMAL', 'BCELLLYMPHOMA6',
    'TREGS', 'FOXP3TREG', 'ITREG', 'TREGSFOXP3', 
    'IMMUNOGLOBULIN', 'IMMUNOGLOBULINS', 'IMMUNOGLOBULINE',
    'NONSTRUCTURALPROTEIN4', 'DOPAMINERGIC', 'HELPERCD4', 'TYPEI',
    'ASTHMA', 'INSULIN', 'ADHESIONMOLECULES', 'ACTIVATINGTRANSCRIPTIONFACTOR',
    'GVHD', 'CTLASSOCIATEDTRANSCRIPTIONFACTORS', 'TYPE17',
    'RIBOFLAVIN', 'MUCOSALASSOCIATEDINVARIANTT', 
    'TREGULATORYCELL1', 'NAIVET', 'TSUPPRESSORINDUCER',
    'BROMOHYDRIN', 'PECIFICTHELPER', 'TSUPPRESSOREFFECTOR',
    'TSUPPRESSORPRECURSOR', 'INCLUDINGCORE', 'GRANULOCYTE',
    'SIGNALINGTRANSDUCERSANDACTIVATORSOFTRANSCRIPTION', 'INFLAMMATORYFACTORS',
    'TSCELLINDUCING', 'GROWTHFACTORS', 'REALLYINTERESTINGNEWGENEE3UBIQUITINLIGASES',
] + list(df_ct['type_key'].unique())

def prep_protein(pr):
    pr = pr.strip()
    pr = re.sub('HIGH$|LOW$|High$|high$|Low$|low$|Hi$|hi$|Lo$|lo$|Dim$|Bright$', '', pr)
    pr = pr.upper().strip()
    
    # Strip out enclosures
    pr = re.sub('\\(.*?\\)', '', pr)
    pr = re.sub('<(SUP|SUB)>', '', pr)
    pr = re.sub('(SUP|SUB)>', '', pr)
    pr = re.sub('<(SUP|SUB)', '', pr)
    
    # Replace greeks, super/subscript sybmols and any others before standardizing
    # on alphanumeric only
    pr = unidecode(pr).upper().strip()
    pr = re.sub('[^0-9A-Z]+', '', pr)

    for e in pr_m_sub:
        pr = pr.replace(e[0], e[1])
    pr = pr.strip()
    
    if pr in pr_m_lkp:
        pr = pr_m_lkp[pr].strip()
        
    if len(pr) <= 1 or pr in pr_m_bl:
        return None
    return pr

In [27]:
#prep_protein('CD4(+)CD25(+)FOXP3(+) ⁺⁻ INTERFERONG )( sd CD103Hi ')
#prep_protein('CD45RADIM')
#prep_protein('retinoic acid receptor-related orphan nuclear receptor gamma')

In [28]:
df_mlt['value_norm'] = df_mlt['value'].apply(prep_protein)
df_mlt['value_norm'].value_counts().head(250)

CD4                7616
FOXP3              4766
CD25               3527
CD8                1851
IFNG               1611
IL10               1542
IL2                1288
IL17               1278
TGFB               1175
IL4                1082
TCR                 727
CD3                 604
IL6                 564
IL12                510
IL                  479
HLA                 449
CTLA4               446
IL21                418
TNFA                407
CD127               394
CD28                364
IL17A               359
PD1                 353
IL22                348
IFN                 306
CXCR5               291
IL5                 282
IgE                 279
IL23                277
CCR4                250
IL13                245
CD39                223
TNF                 222
IL1B                216
IL35                215
STAT3               209
CD45RA              203
CD1D                202
CCR6                192
IDO                 190
IL9                 183
MHCII           

In [29]:
df_mlt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69951 entries, 1 to 453611
Data columns (total 8 columns):
id            69951 non-null int64
start         69951 non-null int64
end           69951 non-null int64
value         69951 non-null object
class         69951 non-null object
value_norm    61064 non-null object
value_lbl     61064 non-null object
value_sym     17340 non-null object
dtypes: int64(3), object(5)
memory usage: 4.8+ MB


### Resolve HGNC Lookups

In [30]:
df_hgnc = pd.read_csv(osp.join(DATA_DIR, 'hgnc_map.csv'))
assert df_hgnc['key'].is_unique
df_hgnc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2917 entries, 0 to 2916
Data columns (total 3 columns):
approved_symbol    2917 non-null object
key                2917 non-null object
value              2917 non-null object
dtypes: object(3)
memory usage: 68.4+ KB


In [31]:
#df_hgnc[df_hgnc['key'].str.contains('V')]

In [32]:
df_mlt['value_lbl'] = df_mlt['value_norm'].map(df_hgnc.set_index('key')['value'])
df_mlt['value_sym'] = df_mlt['value_norm'].map(df_hgnc.set_index('key')['approved_symbol'])
df_mlt['value_lbl'] = df_mlt['value_lbl'].where(df_mlt['value_lbl'].notnull(), df_mlt['value_norm'])
assert df_mlt['value_lbl'].notnull().equals(df_mlt['value_norm'].notnull())
df_mlt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69951 entries, 1 to 453611
Data columns (total 8 columns):
id            69951 non-null int64
start         69951 non-null int64
end           69951 non-null int64
value         69951 non-null object
class         69951 non-null object
value_norm    61064 non-null object
value_lbl     61064 non-null object
value_sym     17341 non-null object
dtypes: int64(3), object(5)
memory usage: 4.8+ MB


### Export

In [33]:
# df_mlt[df_mlt['value_norm'].fillna('').str.contains('GRA')]['value_lbl'].value_counts()

In [34]:
df_mlt[df_mlt['value_lbl'] == 'IFNG'].groupby(['value', 'value_lbl'])\
    .size().rename('count').reset_index().sort_values('count', ascending=False)\
    .rename(columns={'value': 'Original', 'value_lbl': 'Normalized', 'count': 'Frequency'})

,Original,Normalized,Frequency
12,IFN-gamma,IFNG,607
16,IFN-γ,IFNG,401
48,interferon-gamma,IFNG,156
43,interferon gamma,IFNG,55
27,IFNγ,IFNG,53
51,interferon-γ,IFNG,46
24,IFNgamma,IFNG,38
4,IFN)-gamma,IFNG,38
6,IFN)-γ,IFNG,31
37,gamma interferon,IFNG,23


In [35]:
df_exp = df_mlt[df_mlt['value_lbl'].notnull()]
df_exp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61064 entries, 1 to 453611
Data columns (total 8 columns):
id            61064 non-null int64
start         61064 non-null int64
end           61064 non-null int64
value         61064 non-null object
class         61064 non-null object
value_norm    61064 non-null object
value_lbl     61064 non-null object
value_sym     17341 non-null object
dtypes: int64(3), object(5)
memory usage: 4.2+ MB


In [36]:
df_exp = df_mlt[df_mlt['value_lbl'].notnull()]
df_exp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61064 entries, 1 to 453611
Data columns (total 8 columns):
id            61064 non-null int64
start         61064 non-null int64
end           61064 non-null int64
value         61064 non-null object
class         61064 non-null object
value_norm    61064 non-null object
value_lbl     61064 non-null object
value_sym     17341 non-null object
dtypes: int64(3), object(5)
memory usage: 4.2+ MB


In [37]:
# Total compression of vocab:
df_exp['value_lbl'].nunique(), df['value'].nunique()

(5051, 9215)

In [38]:
path = osp.join(DATA_DIR, 'pubmed_abstract_proteins_resolved.csv')
df_exp.to_csv(path, index=False)
path

'data/pubmed_abstract_proteins_resolved.csv'